# Методы компрессии нейронных сетей

## Лекция №1 - Введение в компрессию моделей машинного обучения: Вводная
- Что такое компрессия моделей и зачем она нужна?
- Основные задачи в компрессии моделей машинного обучения
- Ограничения и применимость
- Организационные моменты, как будет проходить курс итд

## ДЗ №1
Разделиться на команды по 4 человека и выбрать модели для оптимизации, сделать базовые замеры метрик производительности
|

## Домашняя работа

В качестве модели выбрана YOLOv8m-cls

Проверяются следующие метрики производительности:
- число параметров модели;
- вес файла модели;
- время инференса;
- целевая метрика.

In [ ]:
!pip install ultralytics==8.0.180

In [2]:
from ultralytics import YOLO
import torch
from torch.nn.utils import prune

### Модель

In [3]:
model = YOLO('./best.pt')

#### Число параметров

In [4]:
model.info()

YOLOv8m-cls summary: 141 layers, 15785146 parameters, 0 gradients


(141, 15785146, 0, 0.0)

#### Вес модели

In [7]:
param_size = 0
for param in model.model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 60.179MB


#### Целевая метрика

In [5]:
metrics = model.val(data='imagenette')

Ultralytics YOLOv8.0.180 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m-cls summary (fused): 103 layers, 15775466 parameters, 0 gradients

Dataset not found ⚠️, missing path /content/datasets/imagenette, attempting download...
100%|██████████| 1.45G/1.45G [00:10<00:00, 145MB/s]
Unzipping /content/datasets/imagenette.zip to /content/datasets/imagenette...: 100%|██████████| 13418/13418 [00:13<00:00, 1010.79file/s]
Dataset download success ✅ (24.9s), saved to /content/datasets/imagenette

train: /content/datasets/imagenette/train... found 9469 images in 10 classes ✅ 
val: /content/datasets/imagenette/val... found 3925 images in 10 classes ✅ 
test: None...
val: Scanning /content/datasets/imagenette/val... 3925 images, 0 corrupt: 100%|██████████| 3925/3925 [00:00<00:00, 4748.09it/s]
val: New cache created: /content/datasets/imagenette/val.cache
               classes   top1_acc   top5_acc:   9%|▉         | 22/246 [00:01<00:19, 11.34it/s]
100%|██████████| 755k/755k [0

In [6]:
print("top1_acc:", metrics.top1)
print("top5_acc:", metrics.top5)

top1_acc: 0.9714649319648743
top5_acc: 0.9989808201789856


#### Инференс

In [8]:
for stage, time in metrics.speed.items():
    print(f'{stage} speed {time:.3f} ms')

preprocess speed 0.115 ms
inference speed 1.911 ms
loss speed 0.001 ms
postprocess speed 0.001 ms
